In [123]:
from flask import Flask, make_response, jsonify
import numpy as np
from flask_cors import CORS
#from flask_compress import Compress
app = Flask(__name__)
#Compress(app)
CORS(app)
from glob import glob
from retrodetect.image_processing import getblockmaxedimage

import argparse
import webbrowser
import os
import re
import retrodetect
import pickle
from datetime import datetime
import numpy as np
from datetime import datetime
import numpy as np
import re


In [124]:
pathtoimgsdir = 'Data'
print("Absolute path to images:")
print(os.path.abspath(pathtoimgsdir))
pathtoimgsdir = os.path.abspath(pathtoimgsdir)
pathtoimgs = sorted(glob(pathtoimgsdir+'/*/'))

Absolute path to images:
c:\Users\cs1sch\PycharmProjects\bee_track\bin\Data


In [125]:
pathtoimgsdir

'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data'

In [126]:
if (len(pathtoimgs)==0):
    print("Failed to find any folders in the path, using base path given as camera folder.")
    pathtoimgs = [pathtoimgsdir]
print("Found the following camera folders:")
pathtoimgs
    

Failed to find any folders in the path, using base path given as camera folder.
Found the following camera folders:


['c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data']

In [127]:
configfilename = pathtoimgsdir+'\config_00_00_00.json'
print(configfilename)

c:\Users\cs1sch\PycharmProjects\bee_track\bin\Data\config_00_00_00.json


#functions for  the cell below
#line 147 - 149 
camera_ids = []
for pti in pathtoimgs:
    camera_ids.append(getorderedcamids(pti))

In [128]:
def getimgfilelist(path,camid=None):
    if camid is not None:
        return sorted(glob('%s/*%s*.np'%(path,camid)))
    else:
        return sorted(glob('%s/*.np'%(path)))

In [129]:
def getorderedcamids(path):
    """
    Returns camera ids with greyscale ones first
    """
    fns = getimgfilelist(path)
    print(fns)
    cam_ids = list(set([getcamfromfilename(fn) for fn in fns if getcamfromfilename(fn) is not None]))
    return [cam_id for cam_id in cam_ids if guesscamtype(path,cam_id)=='greyscale']+[cam_id for cam_id in cam_ids if guesscamtype(path,cam_id)=='colour']


In [130]:
def getcamfromfilename(fn):
    res = re.findall('photo_object_([0-9A-Z]*)_[0-9]{8}_',fn)
    if len(res)==0:
        return None
    else:
        return res[0]

In [131]:
def getorderedcamids(path):
    """
    Returns camera ids with greyscale ones first
    """
    fns = getimgfilelist(path)
    cam_ids = list(set([getcamfromfilename(fn) for fn in fns if getcamfromfilename(fn) is not None]))
    return [cam_id for cam_id in cam_ids if guesscamtype(path,cam_id)=='greyscale']+[cam_id for cam_id in cam_ids if guesscamtype(path,cam_id)=='colour']


In [132]:
def guesscamtypegetscore(fn):
    try:
        photo = pickle.load(open(fn,'rb'))
    except EOFError as e:
        return np.NaN


    img = photo['img']
    if img is None:
        score = np.NaN
        #return np.NaN  
    else:
    #e.g. 0.0001 = greyscale, 0.7 = colour
        score = np.abs(np.mean(img[0:-2:2,0:-2:2]/2+img[2::2,2::2]/2-img[1:-2:2,1:-2:2])/np.mean(img))
    return score
    
def guesscamtype(path,camid):
    score = np.nanmean([guesscamtypegetscore(fn) for fn in getimgfilelist(path,camid)[:50:5]])
    if score<0.02:
        return 'greyscale'
    else:
        return 'colour'


In [133]:
#line 147 - 149 
camera_ids = []
for pti in pathtoimgs:
    camera_ids.append(getorderedcamids(pti))

In [108]:
camera_ids

[['02G14695547']]

In [109]:
pathtoimgs

['c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data']

In [73]:
#@app.route('/filename/<int:cam>/<int:internalcam>/<int:number>')
def filename(cam,internalcam,number):
    fn = getimgfilename(cam,internalcam,number)
    photoitem = np.load(fn,allow_pickle=True) 
    returnst = fn
    print(photoitem['record'])
    if 'estimated_true_triggertimestring' in photoitem['record']:
        returnst = returnst + ' (' + photoitem['record']['estimated_true_triggertimestring'] + ')'
    return jsonify(returnst)

In [110]:
fn = getimgfilename(cam,internalcam,number)
fn

'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_06_43.462287__0000.np'

In [111]:
photoitem = np.load(fn,allow_pickle=True) 
print(photoitem['record'])

{'index': 0, 'endofset': True, 'direction': 0, 'flash': True, 'flashselection': [0, 1, 2, 3], 'triggertime': 1688029603.462157, 'triggertimestring': '20230629_10:06:43.462287'}


In [112]:
photoitem

{'index': 0,
 'record': {'index': 0,
  'endofset': True,
  'direction': 0,
  'flash': True,
  'flashselection': [0, 1, 2, 3],
  'triggertime': 1688029603.462157,
  'triggertimestring': '20230629_10:06:43.462287'},
 'img': array([[14, 14, 14, ...,  7,  6,  5],
        [15, 14, 16, ...,  5,  4,  4],
        [14, 14, 12, ...,  4,  4,  4],
        ...,
        [18, 19, 19, ...,  3,  3,  3],
        [18, 18, 17, ...,  2,  3,  3],
        [18, 19, 19, ...,  2,  2,  2]], dtype=uint8),
 'filename': 'photo_object_02G14695547_20230629_10:06:43.462287__0000.np'}

In [75]:
cam = 0
internalcam = 0
number = 0

In [ ]:
fn = getimgfilename(cam,internalcam,number)
-->

fns = getimgfilelist(path)
fns

In [36]:
def getimgfilename(cam,internalcam,number):
    path = pathtoimgs[cam]
    dts = getdatetimelist(path)    
    if number>=len(dts): return None
    
    try:
        fn = getfnfordatetimeandcamid(pathtoimgs[cam],camera_ids[cam][internalcam],dts[number])
    except IndexError:
        return None
    return fn


In [92]:
fn = getfnfordatetimeandcamid(pathtoimgs[cam],camera_ids[cam][internalcam],dts[number])


In [93]:
fn

'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_06_07.051416.np'

In [89]:
def getdatetimelist(path):
    """
    Returns a list of all unique datetimes in path
    """
    fns = getimgfilelist(path)
    return sorted(list(set([getdatetimefromfilename(fn) for fn in fns if getdatetimefromfilename(fn) is not None])))


In [91]:
dts = getdatetimelist(path)
dts

['20230629_10_06_07',
 '20230629_10_06_43',
 '20230629_10_07_20',
 '20230629_10_47_08',
 '20230629_10_47_09',
 '20230629_10_48_59',
 '20230629_10_49_00',
 '20230629_10_49_02',
 '20230629_10_49_03',
 '20230629_10_49_31',
 '20230629_10_49_32',
 '20230629_10_50_21',
 '20230629_10_50_22',
 '20230629_10_50_24',
 '20230629_10_50_25',
 '20230629_10_50_53',
 '20230629_10_50_54',
 '20230629_10_50_55',
 '20230629_10_50_56',
 '20230629_10_50_57',
 '20230629_10_50_58',
 '20230629_10_50_59',
 '20230629_10_51_00',
 '20230629_10_51_01',
 '20230629_10_51_02',
 '20230629_10_51_03',
 '20230629_10_51_04',
 '20230629_10_51_05',
 '20230629_10_51_06',
 '20230629_10_51_07',
 '20230629_10_51_08',
 '20230629_10_51_09',
 '20230629_10_51_10',
 '20230629_10_51_11',
 '20230629_10_51_27',
 '20230629_10_51_30',
 '20230629_10_51_33',
 '20230629_10_51_36',
 '20230629_10_51_39',
 '20230629_10_51_42',
 '20230629_10_51_45',
 '20230629_10_51_48',
 '20230629_10_51_51',
 '20230629_10_51_54',
 '20230629_10_51_57',
 '20230629

In [76]:
fns = getimgfilelist(path)
fns

['c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_06_07.051416.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_06_43.462287__0000.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_06_43.718456__0001.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_06_43.972485__0002.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_07_20.164206__0003.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_07_20.418653__0004.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_47_08.141304__0000.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_47_08.396931__0001.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\

In [77]:
fn = fns[0]

In [88]:
def getdatetimefromfilename(fn):
    #res = re.findall('photo_object_[0-9A-Z]*_([0-9]{8}_[0-9]{2}:[0-9]{2}:[0-9]{2}.[0-9]{6})_',fn)
    # SC: I do this to match his filename format
    res = re.findall('photo_object_[0-9A-Z]*_([0-9]{8}_[0-9]{2}_[0-9]{2}_[0-9]{2})',fn)
    #res = re.findall('photo_object_([0-9]{8}_[0-9]{2}:[0-9]{2}:[0-9]{2}.[0-9]{6})_',fn)
    #res = re.findall('(_[0-9]{2}_[0-9]{2}_[0-9]{2})', fn)
    if len(res)==0:
        return None
    else:
        #return res[0][1:]
        return res[0] # SC: I change to this

In [87]:
res = re.findall('photo_object_[0-9A-Z]*_([0-9]{8}_[0-9]{2}_[0-9]{2}_[0-9]{2})',fn)
res[0][1:]


'0230629_10_06_07'

In [57]:
fns


[]

In [122]:
res = re.findall('photo_object_[0-9A-Z]*_([0-9]{8}[:|_][0-9]{2}_[0-9]{2}_[0-9]{2}.[0-9]{6})',fn)
res

['20230629_10_06_43.462287']

In [34]:
a = getdatetimefromfilename(fn)
a

'0230629_10_06_07.051416'

In [ ]:
#@app.route('/filename/<int:cam>/<int:internalcam>/<int:number>')
def filename(cam,internalcam,number):
    fn = getimgfilename(cam,internalcam,number)
    photoitem = np.load(fn,allow_pickle=True) 
    returnst = fn
    print(photoitem['record'])
    if 'estimated_true_triggertimestring' in photoitem['record']:
        returnst = returnst + ' (' + photoitem['record']['estimated_true_triggertimestring'] + ')'
    return jsonify(returnst)



In [39]:
def getfnfordatetimeandcamid(path,camid,datetime):
    #fns = glob(f'Data\\folder1\\*{camid}_{datetime}*.np')
    fns = glob('%s/*%s_%s*.np'%(path,camid,datetime))
    if len(fns)==0:
        return None
    else:
        return fns[0]

In [50]:
fns = glob('%s/*%s_%s*.np'%('c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data','02G14695547','0230629_10_06_07.051416'))


In [51]:
fns

[]

In [43]:
camera_ids[0][0]

'02G14695547'

In [45]:
internalcam = 0
number = 0

In [49]:
path

'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data'

In [46]:
dts[0]

'0230629_10_06_07.051416'

In [47]:
fn = getfnfordatetimeandcamid(pathtoimgs[cam],camera_ids[cam][internalcam],dts[number])


In [48]:
fn

In [ ]:
def getimgfilename(cam,internalcam,number):
    path = pathtoimgs[cam]
    dts = getdatetimelist(path)    
    if number>=len(dts): return None
    
    try:
        fn = getfnfordatetimeandcamid(pathtoimgs[cam],camera_ids[cam][internalcam],dts[number])
    except IndexError:
        return None
    return fn


In [ ]:
def getfnfordatetimeandcamid(path,camid,datetime):
    #fns = glob(f'Data\\folder1\\*{camid}_{datetime}*.np')
    fns = glob('%s/*%s_%s*.np'%(path,camid,datetime))
    if len(fns)==0:
        return None
    else:
        return fns[0]

In [142]:
def converttodt(st):

    return datetime.strptime(st.replace('_',':'),'%H:%M:%S')  

In [144]:
converttodt('14:10:10')

datetime.datetime(1900, 1, 1, 14, 10, 10)

In [119]:
fns = getimgfilelist(pathtoimgs[cam])
fns
    

['c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_06_43.462287__0000.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_06_43.718456__0001.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_06_43.972485__0002.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_07_20.164206__0003.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_07_20.418653__0004.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_47_08.141304__0000.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_47_08.396931__0001.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_47_08.650303__0002.np',
 'c:\\Users\\cs1sch\\PycharmProj

In [ ]:
#@app.route('/getindexoftime/<int:cam>/<string:dtstring>') #SC: The button 'GO TO'
def getindexoftime(cam:int ,dtstring):
    fns = getimgfilelist(pathtoimgs[cam])
    targ = converttodt(dtstring) #'20210720_13:58:00.000000')
    gotoNum = np.argmin(np.abs([(converttodt(re.findall('.*_([0-9]{8}_[0-9]{2}[:|_][0-9]{2}[:|_][0-9]{2}.[0-9]{6})__',fn)[0])-targ).total_seconds() for fn in fns]))
    
    targ = converttodt(dtstring) #'13:58:00.000000')
    gotoNum = np.argmin(np.abs([(converttodt(re.findall('.*_([0-9]{2}[:|_][0-9]{2}[:|_][0-9]{2})',fn)[0])-targ).total_seconds() for fn in fns]))
    return json.dumps(int(gotoNum))

In [136]:
fns = getimgfilelist(pathtoimgs[cam])
fns
fn = fns[0]
fn
    

'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\photo_object_02G14695547_20230629_10_06_43.462287__0000.np'

In [146]:
targ

datetime.datetime(1900, 1, 1, 13, 58)

In [148]:
(converttodt(re.findall('.*_([0-9]{2}[:|_][0-9]{2}[:|_][0-9]{2})',fn)[0]) - targ).total_seconds()

-13877.0

In [149]:
gotoNum = np.argmin(np.abs([(converttodt(re.findall('.*_([0-9]{2}[:|_][0-9]{2}[:|_][0-9]{2})',fn)[0])-targ).total_seconds() for fn in fns]))


In [150]:
gotoNum

3449

In [139]:
re.findall('.*_([0-9]{8}_[0-9]{2}[:|_][0-9]{2}[:|_][0-9]{2}.[0-9]{6})__',fn)[0]

'20230629_10_06_43.462287'

In [135]:
targ = converttodt('13:58:00')

In [ ]:



@app.route('/loadpos/<int:cam>/<int:internalcam>/<int:number>')
def loadpos(cam, internalcam, number):
    d = load_data(cam, internalcam, number)
    return json.dumps(d)



In [153]:
beetrackfn = os.path.join(pathtoimgsdir, 'bee_track.json')
beetrackfn

'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\bee_track.json'

In [155]:
import json
data = json.load(open(beetrackfn, 'r'))


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\bee_track.json'

In [156]:

def load_data(cam, internalcam, number):
    beetrackfn = os.path.join(pathtoimgsdir, 'bee_track.json')  # SC: we have no json file
    try:
        data = json.load(open(beetrackfn, 'r'))
    except FileNotFoundError:
        data = {}
    # print(data)
    camst = str(cam)
    # internalcamst = str(internalcam) #not used.
    numberst = str(number)

    if camst not in data:
        return []
    if numberst not in data[camst]:
        return []
    return data[camst][numberst]

In [157]:
def loadpos(cam, internalcam, number):
    d = load_data(cam, internalcam, number)
    return json.dumps(d)

In [159]:
d = load_data(cam, internalcam, number)
d

[]

In [160]:
json.dumps(d)

'[]'

: 